In [2]:
!pip3 install pymysql
!pip3 install pytrends

     |████████████████████████████████| 51kB 2.4MB/s 


In [0]:
import pymysql
from pymysql.cursors import DictCursor
import pandas as pd


connection = pymysql.connect(
    host='kaktusv6.beget.tech',
    user='kaktusv6_hack',
    password='Mamont500',
    db='kaktusv6_hack',
    charset='utf8mb4',
    cursorclass=DictCursor
)

def execute_query(query):
    with connection.cursor() as cursor:
        cursor.execute(query)
        return cursor.fetchall()


def get_symptoms():
    return execute_query("select * from symptoms")


def get_regions():
    return execute_query("select google_name, id from regions")


def get_diseases():
    return execute_query("select * from diseases")

def insert(date, region_id, symptom_id, fact_interest):
    temp_insert = f"""
                        INSERT INTO `interest`(`region_id`, `symptom_id`, `fact_interest`, `date`)
                         VALUES ({region_id}, {symptom_id}, {fact_interest}, '{date}')
                """
    execute_query(temp_insert)


In [0]:
# from connect_db import get_regions, get_symptoms, insert
import pandas as pd
from pytrends.request import TrendReq
import numpy as np

pytrends = TrendReq(hl='ru-RU', tz=360)

kw_list = ["кашель", "насморк", "высокая температура"]
def google_trends(kw=None, year_start=2018, month_start=5, year_old=2019, month_old=5, region='RU'):
    data = pytrends.get_historical_interest(kw, year_start=year_start, month_start=month_start,
                                                 day_start=1, hour_start=0, year_end=year_old,
                                                 month_end=month_old, day_end=1, hour_end=0,
                                                 geo=region, cat=45)
    return data


def moving_average(a, n=3) :
    ret = np.cumsum(a, dtype=float)
    ret[n:] = ret[n:] - ret[:-n]
    return ret[n - 1:] / n


# def interest_processing(date, region_id, symptom_id, fact_interest):
#     pass


def google_process():
    # diseases = pd.DataFrame(get_diseases())
    symptoms = pd.DataFrame(get_symptoms())
    regions = pd.DataFrame(get_regions())

    kw_list = list(symptoms.name.values)

    for region, region_id in regions.values:
        data = google_trends(kw=kw_list,  region=region)
        date_day = data.rolling(window=24).mean().iloc[::24].dropna()

        for sympt in date_day.columns:
            sympt_id = symptoms[symptoms.name == sympt].id.values[0]
            for row in date_day.iterows():

                insert(date=row.index, region_id=region_id, symptom_id=sympt_id, fact_interest=row[sympt])





In [0]:
symptoms = pd.DataFrame(get_symptoms())
regions = pd.DataFrame(get_regions())

In [96]:
regions

,google_name,id
0,RU-PRI,1
1,RU-MOW,2
2,RU-KHA,3
3,RU-SAK,4


In [98]:
kw_list = list(symptoms.name.values)
kw_list

['насморк', 'кашель', 'высокая температура', 'лекарства']

In [99]:
data_year2 = pytrends.get_historical_interest(kw_list, year_start=2018, month_start=5, 
                                 day_start=1, hour_start=0, year_end=2019,
                                 month_end=6, day_end=1, hour_end=0,
                                 geo='RU-SAK', cat=45)

The request failed: Google returned a response with code 500.


In [100]:
data_year2.shape[0]

9392

In [0]:
# data = google_trends(kw=kw_list,  region='RU-PRI')
date_day = data_year2.rolling(window=24).mean().iloc[::24].dropna()

In [0]:
# date_day2 = date_day.rolling(window=250).mean().dropna()

In [0]:
# date_day2.iloc[0:1000]

In [0]:
def insert(date, region_id, symptom_id, fact_interest):
    temp_insert = f"""
                        INSERT INTO `interest`(`region_id`, `symptom_id`, `fact_interest`, `date`)
                         VALUES ({region_id}, {symptom_id}, {fact_interest}, '{date}')
                """
    return(temp_insert)

In [0]:
date_day = date_day.drop('isPartial', axis=1)

In [105]:
date_day.columns

Index(['насморк', 'кашель', 'высокая температура', 'лекарства'], dtype='object')

In [107]:
connection = pymysql.connect(
    host='kaktusv6.beget.tech',
    user='kaktusv6_hack',
    password='Mamont500',
    db='kaktusv6_hack',
    charset='utf8mb4',
    cursorclass=DictCursor
)


with connection.cursor() as cursor:   

  for sympt in date_day.columns:
#   sympt_id = 2
#     print(sympt)
    sympt_id = symptoms[symptoms.name == sympt].id.values[0]
    for row in date_day.iterrows():
    #     print(row[0].date())
          cursor.execute(insert(date=row[0].date(), region_id=4, symptom_id=sympt_id, fact_interest=row[1][0]))
connection.commit()

/usr/local/lib/python3.6/dist-packages/pymysql/cursors.py:329: Warning: (1364, "Field 'id' doesn't have a default value")
  self._do_get_result()


In [91]:
connection = pymysql.connect(
    host='kaktusv6.beget.tech',
    user='kaktusv6_hack',
    password='Mamont500',
    db='kaktusv6_hack',
    charset='utf8mb4',
    cursorclass=DictCursor
)


with connection.cursor() as cursor:   
    
  sympt_id = 4

  for row in date_day.iterrows():

        cursor.execute(insert(date=row[0].date(), region_id=1, symptom_id=sympt_id, fact_interest=row[1][0]))
        
connection.commit()

/usr/local/lib/python3.6/dist-packages/pymysql/cursors.py:329: Warning: (1364, "Field 'id' doesn't have a default value")
  self._do_get_result()


In [56]:
date_day.columns

Index(['насморк', 'кашель', 'высокая температура', 'isPartial'], dtype='object')

In [61]:
symptoms[symptoms.name == date_day.columns[2]].id.values[0]

3